In [3]:
import requests
import pandas as pd
import time
import random
import re
import numpy as np
import _pickle as pickle
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs
from scipy.stats import halfnorm

In [4]:
from itertools import permutations
 
# Get all permutations of length 4
categories = ["Style"]
perm = permutations(["Classic", "Elegant", "Dramatic", "Feminine", "Sexy", "Masculine", "Romantic", "Casual", "Streetwear", "Glam", "Minimalist", "Vintage", "Boho", "Editorial", "Androgynous", "Edgy", "Preppy", "Maximalist"], 4)
styles = []
# Print the obtained permutations
for i in list(perm):
    styles.append(', '.join(i))
# styles = np.array(styles)
# Creating a DF from the style permutations
random.shuffle(styles)
styles = pd.DataFrame(styles, columns=["Profiles"])

In [5]:
styles

,Profiles
0,"Masculine, Casual, Preppy, Glam"
1,"Masculine, Glam, Vintage, Preppy"
2,"Sexy, Feminine, Edgy, Editorial"
3,"Classic, Editorial, Androgynous, Feminine"
4,"Maximalist, Androgynous, Elegant, Masculine"
...,...
73435,"Preppy, Casual, Romantic, Editorial"
73436,"Editorial, Maximalist, Edgy, Elegant"
73437,"Dramatic, Streetwear, Feminine, Editorial"
73438,"Maximalist, Minimalist, Classic, Feminine"


In [6]:
# Creating a DF of the style categories
style_df = pd.DataFrame(columns=categories)
print(style_df)
# Filling in Data
for i in style_df.columns:
    
    # Range of numbers to represent different labels in each category
    style_df[i] = np.random.randint(0,18, styles.shape[0])
    
    # Logic: The numbers represent a specific choice within the categories
    # So your number 1 preferred category will have a one and the least will have 18 etc.

Empty DataFrame
Columns: [Style]
Index: []


In [7]:
style_df

,Style
0,15
1,12
2,8
3,9
4,2
...,...
73435,6
73436,9
73437,2
73438,16


In [8]:
# Joining the two dataframes
final_df = styles.join(style_df)
final_df

,Profiles,Style
0,"Masculine, Casual, Preppy, Glam",15
1,"Masculine, Glam, Vintage, Preppy",12
2,"Sexy, Feminine, Edgy, Editorial",8
3,"Classic, Editorial, Androgynous, Feminine",9
4,"Maximalist, Androgynous, Elegant, Masculine",2
...,...,...
73435,"Preppy, Casual, Romantic, Editorial",6
73436,"Editorial, Maximalist, Edgy, Elegant",9
73437,"Dramatic, Streetwear, Feminine, Editorial",2
73438,"Maximalist, Minimalist, Classic, Feminine",16


In [9]:
with open("profiles.pkl", "wb") as fp:
    pickle.dump(final_df, fp)

In [10]:
with open("profiles.pkl",'rb') as fp:
    df = pickle.load(fp)

In [11]:
final_df.to_csv (r'profiles_dataframe.csv', index = False, header=True)

In [26]:
# Removing the numerical data
df = df[['Profiles']]

In [27]:
df

,Profiles
0,"Masculine, Casual, Preppy, Glam"
1,"Masculine, Glam, Vintage, Preppy"
2,"Sexy, Feminine, Edgy, Editorial"
3,"Classic, Editorial, Androgynous, Feminine"
4,"Maximalist, Androgynous, Elegant, Masculine"
...,...
73435,"Preppy, Casual, Romantic, Editorial"
73436,"Editorial, Maximalist, Edgy, Elegant"
73437,"Dramatic, Streetwear, Feminine, Editorial"
73438,"Maximalist, Minimalist, Classic, Feminine"


In [28]:
from numpy.ma.core import default_fill_value
# Probability dictionary
p = {}
style_types = ["Classic", "Elegant", "Dramatic", "Feminine", "Sexy", "Masculine", 
               "Romantic", "Casual", "Streetwear", "Glam", "Minimalist", "Vintage", 
               "Boho", "Editorial", "Androgynous", "Edgy", "Preppy", "Maximalist"]

p['style'] = [0.11, 0.10, 0.07, 0.13, 0.04, 0.06,
              0.06, 0.04, 0.01, 0.01, 0.10, 0.03,
              0.03, 0.07, 0.02, 0.04, 0.01, 0.07]

# Age (generating random numbers based on half normal distribution)
age = halfnorm.rvs(loc=18,scale=8, size=df.shape[0]).astype(int)
# gender = pd.DataFrame(columns=["Gender"])
gender = []
for i in range(df.shape[0]):
    # Range of numbers to represent different labels in each category
    number = random.randint(0, 2)
    if number == 1:
      gender.append("Female")
    elif number == 2:
      gender.append("Non-Binary")
    else:
      gender.append("Male")
# gender = pd.DataFrame(gender, columns=["Gender"])
rate = halfnorm.rvs(loc=20,scale=50, size=df.shape[0]).astype(int)

In [29]:
df

,Profiles
0,"Masculine, Casual, Preppy, Glam"
1,"Masculine, Glam, Vintage, Preppy"
2,"Sexy, Feminine, Edgy, Editorial"
3,"Classic, Editorial, Androgynous, Feminine"
4,"Maximalist, Androgynous, Elegant, Masculine"
...,...
73435,"Preppy, Casual, Romantic, Editorial"
73436,"Editorial, Maximalist, Edgy, Elegant"
73437,"Dramatic, Streetwear, Feminine, Editorial"
73438,"Maximalist, Minimalist, Classic, Feminine"


In [30]:
# final_categories = [style_types, age, gender, rate]
final_categories = [style_types, age, rate]
# names = ["Style", "Age", "Gender", "Rate"]
names = ["Style", "Age", "Rate"]
combined = dict(zip(names, final_categories))

### Make random values for every category

In [31]:
# Looping through and assigning random values
for name, categories in combined.items():
    if name == 'Age' or name == 'Rate':
        # Generating random ages based on a normal distribution
        df[name] = categories
        
    else:
        # Picking 3 from the list 
        try:
            df[name] = list(np.random.choice(categories, size=(df.shape[0],1,4), p=p[name]))
        except:
            df[name] = list(np.random.choice(categories, size=(df.shape[0],1,4)))
        
        df[name] = df[name].apply(lambda x: list(set(x[0].tolist())))
        

/var/folders/jt/9fgtl63s7gv_96_wct99jv2c0000gn/T/ipykernel_66814/133301745.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = list(np.random.choice(categories, size=(df.shape[0],1,4)))
/var/folders/jt/9fgtl63s7gv_96_wct99jv2c0000gn/T/ipykernel_66814/133301745.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = df[name].apply(lambda x: list(set(x[0].tolist())))
/var/folders/jt/9fgtl63s7gv_96_wct99jv2c0000gn/T/ipykernel_66814/133301745.py:5: SettingWithCopyWarning: 
A value is trying 

In [32]:
df

,Profiles,Style,Age,Rate
0,"Masculine, Casual, Preppy, Glam","[Dramatic, Editorial, Edgy, Masculine]",27,30
1,"Masculine, Glam, Vintage, Preppy","[Minimalist, Romantic, Androgynous]",27,82
2,"Sexy, Feminine, Edgy, Editorial","[Classic, Elegant, Editorial]",25,27
3,"Classic, Editorial, Androgynous, Feminine","[Classic, Romantic, Editorial, Vintage]",24,73
4,"Maximalist, Androgynous, Elegant, Masculine","[Glam, Classic, Androgynous]",21,47
...,...,...,...,...
73435,"Preppy, Casual, Romantic, Editorial","[Glam, Vintage, Edgy, Androgynous]",22,83
73436,"Editorial, Maximalist, Edgy, Elegant","[Casual, Glam, Boho, Elegant]",25,21
73437,"Dramatic, Streetwear, Feminine, Editorial","[Preppy, Elegant, Sexy]",19,82
73438,"Maximalist, Minimalist, Classic, Feminine","[Classic, Edgy, Androgynous]",38,91


In [33]:
mapping= pd.DataFrame(columns=['id', 'index'])

In [34]:
mapping

,id,index


In [35]:
# values from mongo
from mongo_db_integration import get_stylists
from statistics import median

information = get_stylists()
age = 0
count = -1
# print(information)
for stylist in information:
    new_mapping = pd.DataFrame(columns=mapping.columns, index=[count+1])
    count += 1
    new_profile = pd.DataFrame(columns=df.columns, index=[df.index[-1]+1])
    interests = stylist['interests'] 
    rate = stylist['rate']
    _id = stylist['id']
    new_mapping['id'] = _id
    new_mapping['index'] = df.index[-1]+1 
    if not interests or not rate:
        continue
    for interest in interests:
        if interest[0].isnumeric():
            val = interest[:-7]
            val = val.split('-')
            val = list(map(int, val))
            age = round(median(val))
            interests.remove(interest)
    temp = ', '.join([str(interest) for interest in interests])
    new_profile['Profiles'] = temp
    new_profile['Rate'] = rate
    new_profile['Age'] = age
    style_temp = temp.replace(',', '')
    new_profile['Style'] = style_temp
    # print(val)
    df = pd.concat([df, new_profile])
    # print(new_profile['Style'])
    mapping = pd.concat([mapping, new_mapping])

    


[{'interests': [], 'rate': 29, 'id': 'ARkFdcCOngSf8FBg2j15VQK4fas1'}, {'interests': ['Feminine', 'Classic'], 'rate': 25, 'id': 'XZ9paLnfzrgtT21uLsuhFlveCwY2'}, {'interests': ['funky', '18-29 groups', 'mature', 'nice'], 'rate': 25, 'id': '2YbBnz45JfecTzaO5TbU66WyDB33'}, {'interests': ['less than 18', '$25 - $49', 'Casual/Everyday', 'nice'], 'rate': '12', 'id': 'LwGbmdIvbiWGJuAkaFNQUhGL8s03'}]


In [36]:
mapping

,id,index
1,XZ9paLnfzrgtT21uLsuhFlveCwY2,73440
2,2YbBnz45JfecTzaO5TbU66WyDB33,73441
3,LwGbmdIvbiWGJuAkaFNQUhGL8s03,73442


In [38]:
with open("refined_profiles.pkl",'wb') as fp:
    pickle.dump(df, fp)

with open("mapping.pkl", "wb") as fp:
    pickle.dump(mapping, fp)